In [1]:
import torch
import tensorflow as tf
import os
import logging

/home/jikewct/anaconda3/envs/py3.8/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
sys.path.append('.')
sys.path.append('..')
os.chdir('/home/jikewct/public/jikewct/Repos/ml_code')
from  pipeline.models.network import ncsnpp, lora_ncsnpp

from pipeline.models import FlowMatching
from pipeline.models import *
from pipeline.models import model_utils
from pipeline.models.network.net_utils import mark_only_lora_as_trainable


from configs.flow_matching.cifar10_ncsnpp import get_config
config = get_config()

In [3]:
def convert_score_sde_model(model_name, checkpoint, fm_model):
    #model_name = "2_rect_flow"
    model_path = "/home/jikewct/public/jikewct/Model/rect_flow/"
    model_checkpoint = f"{model_path}/{model_name}_{checkpoint}"
    loaded_state = torch.load(model_checkpoint)
    save_dir = os.path.split(model_checkpoint)[0] 
    model = loaded_state['model']
    cvt_model = dict()

    for  key in  model.keys():
        cvt_key = key.replace("module.", "")
        cvt_model[cvt_key] = model[key]
    torch.save(cvt_model , model_path + f"/{model_name}_cvt_model.pth")
    fm_model.network.load_state_dict(cvt_model)
    model_params = fm_model.network.parameters()
    shadow_params = loaded_state['ema']['shadow_params']
    #print(len(model_params), len(shadow_params))
    parameters = [p for p in model_params if p.requires_grad]
    for ema_param , model_param in zip(shadow_params, parameters):
        if model_param.requires_grad:
            #print(f"copy shape:{model_param.data.shape}")
            model_param.data.copy_(ema_param.data)
    torch.save(fm_model.network.state_dict() ,model_path + f"/{model_name}_model_ema.pth")

# from configs.flow_matching.cifar10_ncsnpp import get_config
# config = get_config()
# fm_model = model_utils.create_model(config)
# convert_score_sde_model("2_rect_flow", "checkpoint_6.pth", fm_model)

In [4]:
from configs.flow_matching.afhq_lora_dog_ncsnpp import get_config
config = get_config()
print(config.model.nn_name)
afhq_lora_dog_model = model_utils.create_model(config)

lora_ncsnpp
Number of Parameters in the Score Model: 32704940


In [5]:

loaded_state = torch.load(config.training.model_checkpoint)

In [6]:
print(loaded_state.keys())
print(afhq_lora_dog_model.network.state_dict().keys())

odict_keys(['sigmas', 'all_modules.0.W', 'all_modules.1.weight', 'all_modules.1.bias', 'all_modules.2.weight', 'all_modules.2.bias', 'all_modules.3.weight', 'all_modules.3.bias', 'all_modules.4.GroupNorm_0.weight', 'all_modules.4.GroupNorm_0.bias', 'all_modules.4.Conv_0.weight', 'all_modules.4.Conv_0.bias', 'all_modules.4.Dense_0.weight', 'all_modules.4.Dense_0.bias', 'all_modules.4.GroupNorm_1.weight', 'all_modules.4.GroupNorm_1.bias', 'all_modules.4.Conv_1.weight', 'all_modules.4.Conv_1.bias', 'all_modules.5.GroupNorm_0.weight', 'all_modules.5.GroupNorm_0.bias', 'all_modules.5.Conv_0.weight', 'all_modules.5.Conv_0.bias', 'all_modules.5.Dense_0.weight', 'all_modules.5.Dense_0.bias', 'all_modules.5.GroupNorm_1.weight', 'all_modules.5.GroupNorm_1.bias', 'all_modules.5.Conv_1.weight', 'all_modules.5.Conv_1.bias', 'all_modules.6.GroupNorm_0.weight', 'all_modules.6.GroupNorm_0.bias', 'all_modules.6.Conv_0.weight', 'all_modules.6.Conv_0.bias', 'all_modules.6.Dense_0.weight', 'all_modules.6.

In [ ]:
afhq_lora_dog_model.network.load_state_dict(loaded_state, strict=False)
((afhq_lora_dog_model.network.state_dict()["all_modules.1.weight"] - loaded_state["all_modules.1.weight"]) ** 2).sum()

In [ ]:
mark_only_lora_as_trainable(afhq_lora_dog_model.network)
for name , p in list(afhq_lora_dog_model.network.named_parameters()):
    print(f"{name}: {p.requires_grad}")
    if "lora_" in name:
        print(p, p.shape)

In [ ]:
load_state1 = torch.load("./data/checkpoints/generative_model/flowMatching/ncsnpp-afhq-31000-model")
load_state2 = torch.load("./data/checkpoints/generative_model/flowMatching/lora_ncsnpp-afhq-2000-model")
print(load_state2.keys() - load_state1.keys())
for key in load_state2.keys():
    if key in load_state1.keys():
        diff = torch.sum((load_state1[key] - load_state2[key]) ** 2)
        if diff > 1e-6:
            print(key, diff)
    else :
        print(key)
        print(torch.sum(load_state2[key]**2))